In [6]:
import numpy as np
from numpy import sqrt
import matplotlib.pyplot as plt
import pandas as pd
import scipy.constants as const
from scipy.optimize import curve_fit                        # Funktionsfit:     popt, pcov = curve_fit(func, xdata, ydata) 
from uncertainties import ufloat                            # Fehler:           fehlerwert =  ulfaot(x, err)
from uncertainties import unumpy as unp 
from uncertainties.unumpy import uarray                     # Array von Fehler: fehlerarray =  uarray(array, errarray)
from uncertainties.unumpy import (nominal_values as noms,   # Wert:             noms(fehlerwert) = x
                                  std_devs as stds)         # Abweichung:       stds(fehlerarray) = errarray

Tabelle erstellen:

In [7]:
# md = pd.read_csv('tables/md.csv')
# md_a = md.iloc[:, [0,1]]
# print(md.to_latex(index = False, column_format= "c c", decimal=',')) 

Daten auswerten:

In [8]:
np.savetxt('tables/a.txt', md.values, header='f/Hz U/V', fmt='%.3f')
f, U = np.genfromtxt('tables/a.txt', unpack=True, skip_header=1)

# Ausgleichsfunktion
def g(f, T):
    w = 2 * np.pi * f
    return 1 / (np.sqrt(1 + w**2 * T**2))

para, pcov = curve_fit(g, f, U)
pcov = np.sqrt(np.diag(pcov))
a = para
fa = pcov 

NameError: name 'md' is not defined

Fehler berechnen:

In [ ]:
ua = ufloat(a, fa) 
print('T = (%.3f ± %.3f)' % (noms(ua), stds(ua)))

Plot erstellen:

In [ ]:

plt.plot(f, Uc, 'xr', markersize=6 , label = 'Messdaten', alpha=0.5)
xx = np.linspace(100, 150000, 10**4)
plt.plot(xx, g(xx, a), '-b', linewidth = 1, label = 'Ausgleichsfunktion')

plt.xlabel(r'$f \, / \, \mathrm{Hz}$')
plt.ylabel(r'$A(\omega) \, / \, U_0$')
plt.xscale('log')                       # axis scale 
plt.legend(loc="best")                  # legend position
plt.grid(True)                          # grid style
plt.xlim(200, 150000)                   # limitation of visible scale in plot

Fehler berechnen

In [18]:
N = ufloat(960, 31)     # Zählrate Nullmessung
print('Aktivität: ', noms(N) / 900 , stds(N / 900))

Aktivität:  1.0666666666666667 0.034444444444444444
